In [1]:
%store -r BER_Dublin_specific

In [2]:
import pandas as pd
import numpy as np

In [3]:
def calc_sec_SH_Demand(
    CHPSystemType,
    gsdHSSupplHeatFraction,
    HSSupplHeatFraction,
    delivered_energy_sec,
    delivered_energy_main,
    efficiency,
    CHPUnitHeatFraction,
):

    # Group Heating (i.e. HH uses CHP or waste energy for HW & SH)
    """ EXCEL FORMULA: IF(J8=0,0,J8*SH!H14/(J9/100))
        J8 = Fraction of heat from secondary / supplementary system (from Table 7, Table 10 or Appendix F)
        SH!H14 = Gross heat emission to heated space [kWh/y]
        J9 = Efficiency of secondary / supplementary system [%] (from Table 4a or Appendix E)
        """
    if pd.notnull(CHPSystemType) :
        
        """ delivered_energy_main = total_SH * (1 - frac_used_by_secondary)
            delivered_energy_sec = frac_used_by_secondary * total_SH
                                 = frac_used_by_secondary * delivered_energy_main/(1 - frac_used_by_secondary)
            """
        return (gsdHSSupplHeatFraction/1-gsdHSSupplHeatFraction) * delivered_energy_main * efficiency
   
            
    # Individual Heating 
    """ EXCEL FORMULA: IF(J8=0,0,J8*SH!H14/(J9/100))
        J8 = Fraction of heat from secondary / supplementary system (from Table 7, Table 10 or Appendix F)
        SH!H14 = Gross heat emission to heated space [kWh/y]
        J9 = Efficiency of secondary / supplementary system [%] (from Table 4a or Appendix E)
        """
    if efficiency > 0:
        return HSSupplHeatFraction * delivered_energy_main / efficiency
    else:
        return 0

calc_sec_SH_Demand = np.vectorize(calc_sec_SH_Demand)

In [4]:
BER_Dublin_specific[r"DeliveredEnergySecondarySpace (ind)"] = calc_sec_SH_Demand(
    BER_Dublin_specific["CHPSystemType"].values,
    BER_Dublin_specific["gsdHSSupplHeatFraction_edited"].values,
    BER_Dublin_specific["HSSupplHeatFraction"].values,
    BER_Dublin_specific["DeliveredEnergySecondarySpace"].values,
    BER_Dublin_specific[r"DeliveredEnergyMainSpace (ind)"].values,
    BER_Dublin_specific["HSSupplSystemEff_edited"].values,
    BER_Dublin_specific["CHPUnitHeatFraction"].values,
    
)

In [5]:
BER_Dublin_specific[
    [
        "DeliveredEnergySecondarySpace",
        "DeliveredEnergySecondarySpace (ind)",
        "HSSupplSystemEff_edited",
        "HSSupplHeatFraction",
    ]
].head()

,DeliveredEnergySecondarySpace,DeliveredEnergySecondarySpace (ind),HSSupplSystemEff_edited,HSSupplHeatFraction
2,3440.495,3095.903729,0.30,0.1
6,2116.064,1904.457400,0.72,0.1
15,1608.386,1446.376229,0.63,0.1
18,1891.684,1701.137783,0.40,0.1
22,0.000,0.000000,0.00,0.0


In [6]:
%store BER_Dublin_specific

Stored 'BER_Dublin_specific' (DataFrame)
